In [1]:
import numpy as np
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units

In [2]:
Lv = 2.501*1e6   # [J kg-1] latent heat of vaporization
Rd = 287   # [J K-1 kg-1] gas constant for dry air
Rv = 461   # [J K-1 kg-1] gas constant for water vapor
Cp = 1005.7   # [J kg-1 K-1] specific heat capacity for dry ai

Epsilon = Rd / Rv
print(Epsilon)

0.6225596529284165


In [3]:
T = 20.0
P = 950.0
Qv = 0.010

TK = T + 273.15

In [22]:
# Vapor pressure [hPa]
Ep = P * Qv / (Epsilon + Qv)
Ep

15.01834642159048

In [31]:
Ep_mp = mpcalc.vapor_pressure(P * units('hPa'), Qv)
Ep_mp

15.032670501431127 <Unit('hectopascal')>

In [4]:
# Saturation vapor pressure [hPa]
Es = 6.112 * np.exp(17.67 * T / (T + 243.5))
Es

23.36947123406443

In [5]:
mpcalc.saturation_vapor_pressure(T * units('degC')).to(units('hPa'))

23.36947123406443 <Unit('hectopascal')>

In [6]:
# Saturation mixing ratio [kg/kg]
Qv_sat = Epsilon * Es / (P - Es)
Qv_sat

0.01570085319763383

In [19]:
# RH = 100 * Ep / Es
RH = 100 * Qv / Qv_sat
RH

63.69080631558948

In [8]:
RH_mp = mpcalc.relative_humidity_from_mixing_ratio(P * units.hPa, T * units.degC, Qv).to('percent')
RH_mp

63.7525295295548 <Unit('percent')>

In [75]:
# Potential temperature
Theta = TK * (1000. / P) ** (Rd/Cp)
# Theta = TK * (1000. / (P-Ep)) ** (Rd/Cp)
print(Theta)

297.47261305177165


In [78]:
# mpcalc.potential_temperature(P * units('hPa'), T * units('degC'))
# P - Ep
# mpcalc.potential_temperature((P - Ep) * units('hPa'), T * units('degC'))

In [79]:
Theta_mp = mpcalc.potential_temperature(P * units('hPa'), T * units('degC'))
Theta_mp

297.4778148814758 <Unit('kelvin')>

In [38]:
# Dew point temperature [C]
TD = 243.5 * np.log(Ep / 6.112) / (17.67 - np.log(Ep / 6.112))
# TD = 243.5 * np.log(Ep / 6.1121) / (18.678 - np.log(Ep / 6.1121))
TD

13.0529631726017

In [32]:
TD_mp = mpcalc.dewpoint_from_relative_humidity(T * units('degC'), RH_mp)
TD_mp

12.930590862932661 <Unit('degree_Celsius')>

In [80]:
mpcalc.equivalent_potential_temperature(P * units('hPa'), T * units('degC'), TD_mp).to('degC')

53.35565601467147 <Unit('degree_Celsius')>

In [82]:
# Temperature at lifting condensation level (Eq. 15 in Bolton 1980)
# TL = 1 / (1/(TD - 56) + np.log(T/TD)/800) + 56
TL = 56 + 1. / (1. / (TD - 56) + np.log(T / TD) / 800.)
TL

12.04607573459586

In [84]:
# Dry potential temperature at LCL [K]
# Note the first term is different from Theta calculation, vapor pressure is subtracted from total pressure
ThetaL = TK * (1000. / (P - Ep))**(Rd/Cp) * (TK / (TL+273.15)) ** (0.28 * Qv)

ThetaE = ThetaL * np.exp(Qv * (1 + 0.448 * Qv) * (3036. / (TL+273.15) - 1.78))
ThetaE-273.15

53.53515390033442

In [90]:
# AMS Glossary
# https://glossary.ametsoc.org/wiki/Equivalent_potential_temperature
# ThetaE = Theta * RH ** (-Qv * Rv / Cp) * np.exp(Lv * Qv / (Cp * TK))
ThetaE = TK * (1000 / P)**(Rd/Cp) * RH**(-Qv * Rv / Cp) * np.exp(Lv * Qv / (Cp * TK))
print(ThetaE - 273.15)

44.551328591129504


In [95]:
# Simplified formula by Stull (1988) 
# https://en.wikipedia.org/wiki/Equivalent_potential_temperature
(TK + Qv * Lv / Cp) * (1000 / P)**(Rd/Cp) - 273.15

49.55755624780335

In [96]:
# ThetaE = TK * (1000 / P)**(0.2854 * (1 - 0.28 * Qv)) * np.exp((3.376 / TL - 0.00254) * Qv * (1 + 0.81 * Qv))
# ThetaE - 273.15

In [97]:
Rd/Cp

0.28537337178084915

In [99]:
def theta_e(T, P, Qv):
    """
    Calculate equivalent potential temperature.

    Args:
        T: array-like
            Dry air temperature [degree Celsius]
        P: array-like
            Total air pressure [hPa]
        Qv: array-like
            Vapor mixing ratio [kg/kg]
    Returns:
        ThetaE: array-like
            Equivalent potential temperature [K]
    """
    # Constants
    Lv = 2.501*1e6   # [J kg-1] latent heat of vaporization
    R_dry = 287   # [J K-1 kg-1] gas constant for dry air
    R_v = 461   # [J K-1 kg-1] gas constant for water vapor
    Cp_dry = 1005.7   # [J kg-1 K-1] specific heat capacity for dry air
    Epsilon = R_dry / R_v
    
    # Temperature [K]
    TK = T + 273.15

    # Vapor pressure [hPa]
    Ep = P * Qv / (Epsilon + Qv)

    # Dew point temperature [C]
    TD = 243.5 * np.log(Ep / 6.112) / (17.67 - np.log(Ep / 6.112))

    # Temperature at lifting condensation level (Eq. 15 in Bolton 1980)
    TL = 56 + 1. / (1. / (TD - 56) + np.log(T / TD) / 800.)

    # Dry potential temperature at LCL [K] (Eq. 24 in Bolton 1980)
    ThetaL = TK * (1000. / (P - Ep))**(R_dry/Cp_dry) * (TK / (TL+273.15)) ** (0.28 * Qv)

    # Equivalent potential temperature (Eq. 39 in Bolton 1980)
    ThetaE = ThetaL * np.exp(Qv * (1 + 0.448 * Qv) * (3036. / (TL+273.15) - 1.78))
    
    return ThetaE

In [101]:
theta_e(T, P, Qv) - 273.15

53.53515390033442

In [106]:
# RH_mp = mpcalc.relative_humidity_from_mixing_ratio(P * units.hPa, T * units.degC, Qv).to('percent')

# TD_mp = mpcalc.dewpoint_from_relative_humidity(T * units('degC'), RH_mp)

Ep_mp = mpcalc.vapor_pressure(P * units('hPa'), Qv)

TD_mp = mpcalc.dewpoint(Ep_mp)

mpcalc.equivalent_potential_temperature(P * units('hPa'), T * units('degC'), TD_mp).to('degC')

53.61374726282281 <Unit('degree_Celsius')>